*Read data form bronze layer*

In [0]:
%sql
SELECT * FROM bara_slaes_project.bronze.erp_loc_a101  LIMIT 5;

In [0]:
df=spark.table("bara_slaes_project.bronze.erp_loc_a101")

---
*Data Transformation*

In [0]:
# *Rename Cols*
rename_mapping = {
    'CID':'Customer_ID',
    'CNTRY':'Country'
}

df = df.withColumnsRenamed(rename_mapping)

In [0]:

## Trim
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql.functions import *
for field in df.schema.fields:
    if isinstance(field.dataType,StringType):
        df=df.withColumn(field.name,trim(col(field.name)))

In [0]:
#fill Null Values
df=df.fillna(value="Unknown",subset=["Country"])
#Replace Values
Country_replacements = {
    'CID':'Customer_ID',
    'US':'United States',
    'USA':'United States',
    "DE":"Denmak",
    "":"Unknown"
}
df=df.replace(Country_replacements,subset=["Country"])

In [0]:
# reform the Customer_ID AND CONVERT TO INT TO MATCH THE Customer_ID IN crm_customer_info TABLE
df=(
    df
    .withColumn("Customer_ID", expr("substring(Customer_ID, 6)"))
    .withColumn("Customer_ID", col("Customer_ID").cast(IntegerType()))

)

---
*Write into Silver layer*

In [0]:
df.write.mode("overwrite")\
    .format("delta")\
        .saveAsTable("bara_slaes_project.silver.erp_country_info")

In [0]:
%sql
SELECT * FROM bara_slaes_project.silver.erp_country_info  LIMIT 5;